In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install geopandas

     |████████████████████████████████| 994 kB 5.3 MB/s 
     |████████████████████████████████| 15.4 MB 35 kB/s 
     |████████████████████████████████| 6.3 MB 39.6 MB/s 


In [3]:
import geopandas as gpd
import pandas as pd

In [4]:
df_ppl=gpd.read_file('/content/drive/MyDrive/Colab Notebooks/MCLP_H2 /1KM_인구격자/(B100)국토통계_인구정보-총 인구 수(전체)-(격자) 1KM_서울특별시_202104/nlsp_020001001.shp', encoding = 'utf-8')

In [5]:
# VAL: 격자 내 인구수

df_ppl

,gid,lbl,val,geometry
0,다사6453,9486.00,9486.0,"POLYGON ((964000.000 1953000.000, 964000.000 1..."
1,다사5651,12646.00,12646.0,"POLYGON ((956000.000 1951000.000, 956000.000 1..."
2,다사6157,30514.00,30514.0,"POLYGON ((961000.000 1957000.000, 961000.000 1..."
3,다사5858,24530.00,24530.0,"POLYGON ((958000.000 1958000.000, 958000.000 1..."
4,다사5347,12016.00,12016.0,"POLYGON ((953000.000 1947000.000, 953000.000 1..."
...,...,...,...,...
705,다사4053,None,NaN,"POLYGON ((940000.000 1953000.000, 940000.000 1..."
706,다사7249,None,NaN,"POLYGON ((972000.000 1949000.000, 972000.000 1..."
707,다사4552,None,NaN,"POLYGON ((945000.000 1952000.000, 945000.000 1..."
708,다사6265,None,NaN,"POLYGON ((962000.000 1965000.000, 962000.000 1..."


In [6]:
df_ppl = df_ppl.to_crs(epsg=4329)
df_ppl.head()

,gid,lbl,val,geometry
0,다사6453,9486.00,9486.0,"POLYGON ((127.09230 37.57567, 127.09225 37.584..."
1,다사5651,12646.00,12646.0,"POLYGON ((127.00183 37.55730, 127.00177 37.566..."
2,다사6157,30514.00,30514.0,"POLYGON ((127.05812 37.61161, 127.05806 37.620..."
3,다사5858,24530.00,24530.0,"POLYGON ((127.02407 37.62049, 127.02401 37.629..."
4,다사5347,12016.00,12016.0,"POLYGON ((126.96812 37.52110, 126.96805 37.530..."


In [7]:
#Pydeck 사용을 위한 함수 정의
import geopandas as gpd 
import shapely # Shapely 형태의 데이터를 받아 내부 좌표들을 List안에 반환합니다. 
def line_string_to_coordinates(line_string): 
    if isinstance(line_string, shapely.geometry.linestring.LineString): 
        lon, lat = line_string.xy 
        return [[x, y] for x, y in zip(lon, lat)] 
    elif isinstance(line_string, shapely.geometry.multilinestring.MultiLineString): 
        ret = [] 
        for i in range(len(line_string)): 
            lon, lat = line_string[i].xy 
            for x, y in zip(lon, lat): 
                ret.append([x, y])
        return ret 

# def multipolygon_to_coordinates(x): 
#     lon, lat = x[0].exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 

def polygon_to_coordinates(x): 
    lon, lat = x.exterior.xy 
    return [[x, y] for x, y in zip(lon, lat)] 


In [8]:
# val 열 na 제거
df_ppl['val'] = df_ppl['val'].fillna(0)

# 인구 수 정규화
df_ppl['정규화인구'] = df_ppl['val'] / df_ppl['val'].max()

# geometry를 coordinate 형태로 적용
df_ppl['coordinates'] = df_ppl['geometry'].apply(polygon_to_coordinates)

In [9]:
df_ppl['geometry'] 

0      POLYGON ((127.09230 37.57567, 127.09225 37.584...
1      POLYGON ((127.00183 37.55730, 127.00177 37.566...
2      POLYGON ((127.05812 37.61161, 127.05806 37.620...
3      POLYGON ((127.02407 37.62049, 127.02401 37.629...
4      POLYGON ((126.96812 37.52110, 126.96805 37.530...
                             ...                        
705    POLYGON ((126.82052 37.57442, 126.82044 37.583...
706    POLYGON ((127.18305 37.53990, 127.18302 37.548...
707    POLYGON ((126.87722 37.56572, 126.87714 37.574...
708    POLYGON ((127.06903 37.68375, 127.06898 37.692...
709    POLYGON ((126.95602 37.62921, 126.95595 37.638...
Name: geometry, Length: 710, dtype: geometry

In [10]:
# 100X100 grid에서 central point 찾기
df_ppl_list = []
df_ppl_list2 = []
for i in df_ppl['geometry']:
    cent = [[i.centroid.coords[0][0],i.centroid.coords[0][1]]]
    df_ppl_list.append(cent)
    df_ppl_list2.append(i.centroid)
df_ppl['coord_cent'] = 0
df_ppl['geo_cent'] = 0
df_ppl['coord_cent']= pd.DataFrame(df_ppl_list) # pydeck을 위한 coordinate type
df_ppl['geo_cent'] = df_ppl_list2 # geopandas를 위한 geometry type

In [11]:
df_ppl

,gid,lbl,val,geometry,정규화인구,coordinates,coord_cent,geo_cent
0,다사6453,9486.00,9486.0,"POLYGON ((127.09230 37.57567, 127.09225 37.584...",0.206811,"[[127.0923035430468, 37.57567440305103], [127....","[127.09794163380136, 37.58020043829302]",POINT (127.0979416338014 37.58020043829302)
1,다사5651,12646.00,12646.0,"POLYGON ((127.00183 37.55730, 127.00177 37.566...",0.275704,"[[127.00182724968018, 37.55730030462514], [127...","[127.00745844605304, 37.561830628142914]",POINT (127.007458446053 37.56183062814291)
2,다사6157,30514.00,30514.0,"POLYGON ((127.05812 37.61161, 127.05806 37.620...",0.665257,"[[127.0581165973122, 37.61160520068249], [127....","[127.06375529184683, 37.61613284538466]",POINT (127.0637552918468 37.61613284538466)
3,다사5858,24530.00,24530.0,"POLYGON ((127.02407 37.62049, 127.02401 37.629...",0.534796,"[[127.02406915802318, 37.62048620853416], [127...","[127.0297064357866, 37.625015464100706]",POINT (127.0297064357866 37.62501546410071)
4,다사5347,12016.00,12016.0,"POLYGON ((126.96812 37.52110, 126.96805 37.530...",0.261969,"[[126.96811838272411, 37.52109923832573], [126...","[126.97374482409313, 37.52563116149439]",POINT (126.9737448240931 37.52563116149439)
...,...,...,...,...,...,...,...,...
705,다사4053,None,0.0,"POLYGON ((126.82052 37.57442, 126.82044 37.583...",0.000000,"[[126.82051991631344, 37.57442288713654], [126...","[126.82614128007928, 37.57896175706814]",POINT (126.8261412800793 37.57896175706814)
706,다사7249,None,0.0,"POLYGON ((127.18305 37.53990, 127.18302 37.548...",0.000000,"[[127.18305405081533, 37.539898769520654], [12...","[127.18869498499079, 37.544420511791216]",POINT (127.1886949849908 37.54442051179122)
707,다사4552,None,0.0,"POLYGON ((126.87722 37.56572, 126.87714 37.574...",0.000000,"[[126.87721510590382, 37.565722190145515], [12...","[126.88283930295496, 37.57025839209927]",POINT (126.882839302955 37.57025839209927)
708,다사6265,None,0.0,"POLYGON ((127.06903 37.68375, 127.06898 37.692...",0.000000,"[[127.0690301996547, 37.683752563572355], [127...","[127.0746749769675, 37.688279668672436]",POINT (127.0746749769675 37.68827966867244)


In [12]:
# 쉬운 분석을 위한 임의의 grid id 부여
df_ppl['grid_id']=0
idx = []
for i in range(len(df_ppl)):
    idx.append(str(i).zfill(5))
df_ppl['grid_id'] = pd.DataFrame(idx)

# 인구 현황이 가장 높은 위치
df_ppl.iloc[df_ppl["val"].sort_values(ascending=False).index].reindex().head()

,gid,lbl,val,geometry,정규화인구,coordinates,coord_cent,geo_cent,grid_id
423,다사6748,45868.00,45868.0,"POLYGON ((127.12650 37.53072, 127.12646 37.539...",1.000000,"[[127.126502533738, 37.53071988230903], [127.1...","[127.13213934415279, 37.535244311861085]",POINT (127.1321393441528 37.53524431186109),00423
38,다사4248,44816.00,44816.0,"POLYGON ((126.84356 37.52949, 126.84348 37.538...",0.977065,"[[126.84356325014052, 37.52948610877284], [126...","[126.84918269396793, 37.534023897319116]",POINT (126.8491826939679 37.53402389731912),00038
559,다사4853,44374.00,44374.0,"POLYGON ((126.91111 37.57491, 126.91104 37.583...",0.967428,"[[126.91111253861166, 37.574909578300165], [12...","[126.91673949704622, 37.57944418152746]",POINT (126.9167394970462 37.57944418152746),00559
168,다사4856,42797.00,42797.0,"POLYGON ((126.91090 37.60195, 126.91083 37.610...",0.933047,"[[126.91089946451756, 37.60194873994623], [126...","[126.91652842434452, 37.60648334954957]",POINT (126.9165284243445 37.60648334954957),00168
205,다사5756,41801.00,41801.0,"POLYGON ((127.01286 37.60241, 127.01280 37.611...",0.911333,"[[127.01285544099093, 37.60241362270109], [127...","[127.01849068052607, 37.60694341377813]",POINT (127.0184906805261 37.60694341377813),00205


In [13]:
pip install pydeck

     |████████████████████████████████| 4.3 MB 5.4 MB/s 
     |████████████████████████████████| 123 kB 48.2 MB/s 
     |████████████████████████████████| 787 kB 29.3 MB/s 
     |████████████████████████████████| 370 kB 48.2 MB/s 
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 1.0.18
    Uninstalling prompt-toolkit-1.0.18:
      Successfully uninstalled prompt-toolkit-1.0.18
  Attempting uninstall: ipython
    Found existing installation: ipython 5.5.0
    Uninstalling ipython-5.5.0:
      Successfully uninstalled ipython-5.5.0
  Attempting uninstall: ipykernel
    Found existing installation: ipykernel 4.10.1
    Uninstalling ipykernel-4.10.1:
      Successfully uninstalled ipykernel-4.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-too

In [14]:

import pydeck as pdk


In [17]:
# Make layer
# 사람이 있는 그리드만 추출
layer = pdk.Layer('PolygonLayer', # 사용할 Layer 타입 
                  df_ppl[(df_ppl['val'].isnull()==False) & df_ppl['val']!=0], # 시각화에 쓰일 데이터프레임 
                  get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름 
                  get_fill_color='[0, 255*정규화인구, 0, 정규화인구*10000 ]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
                  pickable=True, # 지도와 interactive 한 동작 on 
                  auto_highlight=True # 마우스 오버(hover) 시 박스 출력 
                 ) 

# Set the viewport location 
center = [127.0076, 37.5437] 
view_state = pdk.ViewState( 
    longitude=center[0], 
    latitude=center[1], 
    zoom=10
) 

# Render 
# r = pdk.Deck(layers=[layer], initial_view_state=view_state,
#              map_style='mapbox://styles/mapbox/outdoors-v11',
#              mapbox_key = MAPBBOX_API_KEY 
            # )

# r.to_html()


r = pdk.Deck(layers= [layer], initial_view_state=view_state, 
             api_keys = {'mapbox' : "pk.eyJ1Ijoia2tra2trYW4iLCJhIjoiY2tzemcwMmRvMnRsaTMxbnY0MjlidjU4NCJ9.iJ2LgGMqRh1WLDPQ5epWyg"},
             map_style='road')

r.to_html()

<IPython.core.display.Javascript object>

'<!DOCTYPE html>\n<html>\n  <head>\n    <meta http-equiv="content-type" content="text/html; charset=UTF-8" />\n    <title>pydeck</title>\n        <script src="https://api.tiles.mapbox.com/mapbox-gl-js/v1.13.0/mapbox-gl.js"></script>\n        <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/bootstrap/3.2.0/css/bootstrap-theme.min.css" />\n    <link rel="stylesheet" href="https://maxcdn.bootstrapcdn.com/font-awesome/4.6.3/css/font-awesome.min.css" />\n    <script src=\'https://cdn.jsdelivr.net/npm/@deck.gl/jupyter-widget@~8.5.*/dist/index.js\'></script>\n    <style>\n    body {\n  margin: 0;\n  padding: 0;\n  overflow: hidden;\n}\n\n#deck-map-container {\n  width: 100%;\n  height: 100%;\n  background-color: black;\n}\n\n#map {\n  pointer-events: none;\n  height: 100%;\n  width: 100%;\n  position: absolute;\n  z-index: 1;\n}\n\n#deckgl-overlay {\n  z-index: 2;\n  background: none;\n}\n\n#deck-map-wrapper {\n  width: 100%;\n  height: 100%;\n}\n\n#deck-container {\n  width: 100v

In [ ]:
pip install shapely

In [ ]:
def generate_candidate_sites(points,M=100):
    '''
    Generate M candidate sites with the convex hull of a point set
    Input:
        points: a Numpy array with shape of (N,2)
        M: the number of candidate sites to generate
    Return:
        sites: a Numpy array with shape of (M,2)
    '''
    hull = ConvexHull(points)
    polygon_points = points[hull.vertices]
    poly = Polygon(polygon_points)
    min_x, min_y, max_x, max_y = poly.bounds
    sites = []
    while len(sites) < M:
        random_point = Point([random.uniform(min_x, max_x),
                             random.uniform(min_y, max_y)])
        if (random_point.within(poly)):
            sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])




def generate_candidate_sites(df_result_fin,M=100):
    from shapely.geometry import Polygon, Point
    sites = []
    idx=np.random.choice(np.array(range(0,len(df_result_fin))), M)
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.iloc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.iloc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])

def generate_candidate_sites(df_result_fin,Weight,M=100):
    sites = []
    idx = df_result_fin.sort_values(by = Weight, ascending = False).iloc[1:M].index
    for i in range(len(idx)):
        random_point = Point(np.array(df_result_fin.loc[idx]['coord_cent'])[i][0],
                             np.array(df_result_fin.loc[idx]['coord_cent'])[i][1])
        sites.append(random_point)
    return np.array([(p.x,p.y) for p in sites])



from scipy.spatial import distance_matrix
def mclp(points,K,radius,M,df_result_fin,w,Weight):

    """
    Solve maximum covering location problem
    Input:
        points: input points, Numpy array in shape of [N,2]
        K: the number of sites to select
        radius: the radius of circle
        M: the number of candidate sites, which will randomly generated inside
        the ConvexHull wrapped by the polygon
    Return:
        opt_sites: locations K optimal sites, Numpy array in shape of [K,2]
        f: the optimal value of the objective function
    """
    print('----- Configurations -----')
    print('  Number of points %g' % points.shape[0])
    print('  K %g' % K)
    print('  Radius %g' % radius)
    print('  M %g' % M)
    import time
    start = time.time()
    sites = generate_candidate_sites(df_result_fin,Weight,M)
    J = sites.shape[0]
    I = points.shape[0]
    D = distance_matrix(points,sites)
    mask1 = D<=radius
    D[mask1]=1
    D[~mask1]=0

    from mip import Model, xsum, maximize, BINARY

    # Build model
    m = Model("mclp")
    # Add variables

    x = [m.add_var(name = "x%d" % j, var_type = BINARY) for j in range(J)]
    y = [m.add_var(name = "y%d" % i, var_type = BINARY) for i in range(I)]


    m.objective = maximize(xsum(w[i]*y[i] for i in range (I)))

    m += xsum(x[j] for j in range(J)) == K

    for i in range(I):
        m += xsum(x[j] for j in np.where(D[i]==1)[0]) >= y[i]

    m.optimize()
    
    end = time.time()
    print('----- Output -----')
    print('  Running time : %s seconds' % float(end-start))
    print('  Optimal coverage points: %g' % m.objective_value)

    solution = []
    for i in range(J):
        if x[i].x ==1:
            solution.append(int(x[i].name[1:]))
    opt_sites = sites[solution]
            
    return opt_sites,m.objective_value


